In [1]:
import numpy as np
import cv2
from skimage.transform import hough_ellipse

res_fac = 1

img = cv2.imread('img/raw/4.png')
h, w, _ = img.shape

img_resized = cv2.resize(img, (w // res_fac, h // res_fac))
blurred = cv2.blur(img_resized, (7, 7))
hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

# mask = cv2.inRange(hsv, (0, 0, 67), (255, 255, 255))
mask = cv2.inRange(hsv, (37, 80, 0), (255, 255, 255))
mask = 255 - mask

kernel = np.ones((5, 5), np.uint8)
morph = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

connectivity = 4
output = cv2.connectedComponentsWithStats(morph, connectivity, cv2.CV_32S)

#non max suppression

num_labels = output[0]
labels = output[1]
stats = output[2]

filtered = np.zeros_like(morph)

for i in range(1, num_labels):
    a = stats[i, cv2.CC_STAT_AREA]
    top = stats[i, cv2.CC_STAT_TOP]
    left = stats[i, cv2.CC_STAT_LEFT]
    width = stats[i, cv2.CC_STAT_WIDTH]
    height = stats[i, cv2.CC_STAT_HEIGHT]
    
    if (a > 1500):
        filtered[np.where(labels == i)] = 255
        
        # cv2.putText(img_resized, str(a), (left, top), cv2.FONT_ITALIC, 1, (0, 0, 255), 2, cv2.LINE_4)
        # cv2.rectangle(img_resized, (left, top), (left + width, top + height), (0, 255, 0), 1)



gray = cv2.cvtColor(filtered, cv2.COLOR_GRAY2BGR)
gray = cv2.cvtColor(gray, cv2.COLOR_BGR2GRAY)

cv2.namedWindow('gray', cv2.WINDOW_KEEPRATIO)
cv2.imshow('gray', gray)
cv2.resizeWindow('gray', w // 2, h // 2)

# ellipsis = hough_ellipse(gray, accuracy=20, threshold=250, min_size=100, max_size=120)
# ellipsis.sort(order='accumulator')

# best = list(ellipsis[-1])
# yc, xc, a, b = [int(round(x)) for x in best[1:5]]
# orientation = best[5]

# cv2.ellipse(img_resized, (xc, yc), (a, b), np.degrees(orientation), 0, 360, (0, 255, 0), 2)

circles = cv2.HoughCircles(gray, cv2.HOUGH_STANDARD, dp=1.5, minDist=80, minRadius=100)
# circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT_ALT, dp=1.5, minDist=20, param1=100, param2=0.5, minRadius=50)

if (circles is not None):
        for i in circles[0,:]:
            # draw the outer circle
            cv2.circle(img_resized,(int(i[0]), int(i[1])),int(i[2]), (0,255,0), 2)
            # draw the center of the circle
            cv2.circle(img_resized,(int(i[0]), int(i[1])), 2, (0,0,255), 3)
            
if circles is not None:
    circles = np.round(circles[0, :]).astype("int")
    for (x, y, r) in circles:
        cv2.circle(img_resized, (x, y), r, (0, 255, 0), 2)
            
else:
    print("No circles")


cv2.namedWindow('custom window', cv2.WINDOW_KEEPRATIO)
cv2.imshow('custom window', img_resized)
cv2.resizeWindow('custom window', w // 2, h // 2)

cv2.namedWindow('custom mask window', cv2.WINDOW_KEEPRATIO)
cv2.imshow('custom mask window', mask)
cv2.resizeWindow('custom window', w // 2, h // 2)

cv2.namedWindow('custom filt mask window', cv2.WINDOW_KEEPRATIO)
cv2.imshow('custom filt mask window', filtered)
cv2.resizeWindow('custom window', w // 2, h // 2)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(10)



error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\hough.cpp:2338: error: (-5:Bad argument) Unrecognized method id. Actually supported methods are HOUGH_GRADIENT and HOUGH_GRADIENT_ALT in function 'cv::HoughCircles'


In [1]:
import cv2
import numpy as np

res_fac = 2

cv2.namedWindow('mask')

def nothing(x):
    pass

cv2.createTrackbar('lh', 'mask', 0, 255, nothing)
cv2.createTrackbar('ls', 'mask', 0, 255, nothing)
cv2.createTrackbar('lv', 'mask', 64, 255, nothing)
cv2.createTrackbar('hh', 'mask', 255, 255, nothing)
cv2.createTrackbar('hs', 'mask', 255, 255, nothing)
cv2.createTrackbar('hv', 'mask', 255, 255, nothing)

img = cv2.imread('img/mask/1.png')
h, w, _ = img.shape
img_resized = cv2.resize(img, (w // res_fac, h // res_fac))


hsv = cv2.cvtColor(img_resized, cv2.COLOR_BGR2HSV)

while True:
    lh = cv2.getTrackbarPos('lh', 'mask')
    ls = cv2.getTrackbarPos('ls', 'mask')
    lv = cv2.getTrackbarPos('lv', 'mask')
    hh = cv2.getTrackbarPos('hh', 'mask')
    hs = cv2.getTrackbarPos('hs', 'mask')
    hv = cv2.getTrackbarPos('hv', 'mask')

    mask = cv2.inRange(hsv, (lh, ls, lv), (hh, hs, hv))

    mask = 255 - mask
    
    cv2.imshow('mask', mask)

    connectivity = 4  
    output = cv2.connectedComponentsWithStats(mask, connectivity, cv2.CV_32S)

    num_labels = output[0]
    labels = output[1]
    stats = output[2]

    filtered = np.zeros_like(mask)

    for i in range(1, num_labels):
        a = stats[i, cv2.CC_STAT_AREA]
        top = stats[i, cv2.CC_STAT_TOP]
        left = stats[i, cv2.CC_STAT_LEFT]
        width = stats[i, cv2.CC_STAT_WIDTH]
        height = stats[i, cv2.CC_STAT_HEIGHT]
        #print(a)
        
        if (a >= 50):
            filtered[np.where(labels == i)] = 255
            
            cv2.putText(img_resized, str(a), (left, top), cv2.FONT_ITALIC, 1, (0, 0, 255), 2, cv2.LINE_4)
            cv2.rectangle(img_resized, (left, top), (left + width, top + height), (0, 255, 0), 1)


    cv2.imshow('img', img_resized)
    #cv2.imshow('hsv', hsv[:, :, 0])
    cv2.imshow('mask_filtered', filtered)

    key = cv2.waitKey(80) & 0xFF
    
    if (key == ord('q')):
        break
    
    
cv2.destroyAllWindows()
cv2.waitKey(10)

-1

In [1]:
import numpy as np
import cv2

res_fac = 1

img = cv2.imread('img/raw/4.png')
h, w, _ = img.shape

img_resized = cv2.resize(img, (w // res_fac, h // res_fac))

gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)

cv2.namedWindow('gray', cv2.WINDOW_KEEPRATIO)
cv2.imshow('gray', gray)
cv2.resizeWindow('gray', w // 2, h // 2)

def nothing(x):
    pass

cv2.createTrackbar('dp', 'gray', 105, 300, nothing)
cv2.createTrackbar('minDist', 'gray', 10, 200, nothing)
cv2.createTrackbar('param1', 'gray', 100, 500, nothing)
cv2.createTrackbar('param2', 'gray', 65, 100, nothing)
cv2.createTrackbar('minRadius', 'gray', 100, 500, nothing)

while (True):
    
    
    
    dp_t = cv2.getTrackbarPos('dp', 'gray') / 100
    minDist_t = cv2.getTrackbarPos('minDist', 'gray')
    param1_t = cv2.getTrackbarPos('param1', 'gray')
    param2_t = cv2.getTrackbarPos('param2', 'gray') / 100
    minRadius_t = cv2.getTrackbarPos('minRadius', 'gray')
    
    output_img = img_resized.copy()

    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT_ALT, dp=dp_t, minDist=minDist_t, param1=param1_t, 
                               param2=param2_t, minRadius=minRadius_t)

    if (circles is not None):
            for i in circles[0,:]:
                cv2.circle(output_img,(int(i[0]), int(i[1])),int(i[2]), (0,255,0), 2)
                cv2.circle(output_img,(int(i[0]), int(i[1])), 2, (0,0,255), 3)
                
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for (x, y, r) in circles:
            cv2.circle(img_resized, (x, y), r, (0, 255, 0), 2)
                
    else:
        print("No circles")

    cv2.namedWindow('img', cv2.WINDOW_KEEPRATIO)
    cv2.imshow('img', output_img)
    cv2.resizeWindow('img', w // 2, h // 2)

    key = cv2.waitKey(300) & 0xFF
    
    if (key == ord('q')):
        break
    
cv2.destroyAllWindows()
cv2.waitKey(10)



In [13]:
import numpy as np
import cv2
from skimage import color, feature
from skimage.transform import hough_circle, hough_circle_peaks

# Video file path
video_path = 'input.mp4'

# Read the video
cap = cv2.VideoCapture(video_path)

res_fac = 1

cv2.namedWindow('custom window', cv2.WINDOW_KEEPRATIO)

def nothing(x):
    pass

cv2.createTrackbar('lh', 'custom window', 37, 255, nothing)
cv2.createTrackbar('ls', 'custom window', 76, 255, nothing)
cv2.createTrackbar('lv', 'custom window', 0, 255, nothing)
cv2.createTrackbar('hh', 'custom window', 255, 255, nothing)
cv2.createTrackbar('hs', 'custom window', 255, 255, nothing)
cv2.createTrackbar('hv', 'custom window', 255, 255, nothing)

while (True):
    ret, frame = cap.read()
    if not ret:
        break
    
    h, w, _ = frame.shape
    img_resized = cv2.resize(frame, (w // res_fac, h // res_fac))
    blurred = cv2.blur(img_resized, (11, 11))
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    
    lh = cv2.getTrackbarPos('lh', 'custom window')
    ls = cv2.getTrackbarPos('ls', 'custom window')
    lv = cv2.getTrackbarPos('lv', 'custom window')
    hh = cv2.getTrackbarPos('hh', 'custom window')
    hs = cv2.getTrackbarPos('hs', 'custom window')
    hv = cv2.getTrackbarPos('hv', 'custom window')
    
    
    mask = cv2.inRange(hsv, (lh, ls, lv), (hh, hs, hv))
    mask = 255 - mask

    kernel = np.ones((7, 7), np.uint8)
    morph = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    connectivity = 4
    output = cv2.connectedComponentsWithStats(morph, connectivity, cv2.CV_32S)

    num_labels = output[0]
    labels = output[1]
    stats = output[2]

    filtered = np.zeros_like(morph)

    for i in range(1, num_labels):
        a = stats[i, cv2.CC_STAT_AREA]
        
        if a > 5000:
            filtered[np.where(labels == i)] = 255

    circles = cv2.HoughCircles(filtered, cv2.HOUGH_GRADIENT_ALT, dp=4, minDist=20, param1=100, param2=0.5, minRadius=50)

    if circles is not None:
        max_radius = 0
        max_circle = None
        
        for i in circles[0, :]:
            if (i[2] > max_radius):
                max_radius = i[2]
                max_circle = i
        
        if max_circle is not None:
            center_x = int(max_circle[0])
            center_y = int(max_circle[1])
            radius = int(max_circle[2])
            
            cv2.circle(img_resized, (center_x, center_y), radius, (0, 255, 0), 2)
            cv2.circle(img_resized, (center_x, center_y), 2, (0, 0, 255), 3)
            
            text = f"Center: ({center_x}, {center_y}), Radius: {radius}"
            cv2.putText(img_resized, text, (10, 30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 2, cv2.LINE_AA)


    else:
        print("No circles")

    cv2.namedWindow('custom window', cv2.WINDOW_KEEPRATIO)
    cv2.imshow('custom window', img_resized)
    cv2.resizeWindow('custom window', w // 2, h // 2)

    cv2.namedWindow('custom mask window', cv2.WINDOW_KEEPRATIO)
    cv2.imshow('custom mask window', mask)
    cv2.resizeWindow('custom mask window', w // 2, h // 2)

    cv2.namedWindow('custom filt mask window', cv2.WINDOW_KEEPRATIO)
    cv2.imshow('custom filt mask window', filtered)
    cv2.resizeWindow('custom filt mask window', w // 2, h // 2)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(10)


No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles
No circles

-1